In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import random

f = open('history.dat', 'rb')
history_arr = pickle.load(f)


In [ ]:
def make_chart_arr():
    
    tm = []
    val = []
    
    for v in history_arr:
        open =  float(v[1])
        high =  float(v[2])
        low =   float(v[3])
        close = float(v[4])
        
        start_time = v[0]
        end_time = v[6]
        
        up_rotated = True
        if(len(val)>=4):
            up_rotated = open <= val[-4]
        
        tm.append(start_time)
        val.append(open)

        line_points = 5
        
        if up_rotated:
            for j in range(1,line_points):
                val.append(open+(high-open)*j/line_points)
                
            val.append(high)
            
            for j in range(1,line_points):
                val.append(high+(low-high)*j/line_points)
                
            val.append(low)
        
            for j in range(1,line_points):
                val.append(low+(close-low)*j/line_points)
        else:
            for j in range(1,line_points):
                val.append(open+(low-open)*j/line_points)
                
            val.append(low)
            
            for j in range(1,line_points):
                val.append(low+(high-low)*j/line_points)
                
            val.append(high)
        
            for j in range(1,line_points):
                val.append(high+(close-high)*j/line_points)
        
        for j in range(1,line_points*3):
            tm.append(int(start_time+(end_time-start_time)*j/(line_points*3)) )

        tm.append(end_time)
        val.append(close)
    
    return [tm,val]






In [ ]:
CHART_PER_DAY = 24*60

day_arr = []

def make_day_arr():
    global day_arr
    
    day_arr = []
    d = []
    
    for v in history_arr:
        open =  float(v[1])
        high =  float(v[2])
        low =   float(v[3])
        close = float(v[4])
        
        start_time = v[0]
        start_time = int(start_time/(CHART_PER_DAY*1000))*CHART_PER_DAY*1000;
        
        if len(d) != 0 and start_time > d[0]:
            day_arr.append(d)
            d = []
            
        if len(d) == 0:
            d = [start_time,open,high,low,close]
        else:
            d[4] = close
            d[2] = max(d[2],high)
            d[3] = min(d[3],low)


In [ ]:
def findDateIndex(date):
    unix_time = int(datetime.datetime.timestamp(date)*1000)
    index = 0
    for v in history_arr:
        if v[0]>=unix_time:
            break
        index += 1;
    return index
    
from_index = findDateIndex(datetime.datetime(2022, 6, 14, 0, 0,0))
#from_index = findDateIndex(datetime.datetime(2022, 7, 1, 0, 0,0))
#to_index = findDateIndex(datetime.datetime(2022, 6, 14, 0, 0,0))

#history_arr = history_arr[from_index:to_index]
#history_arr = history_arr[from_index:]

make_day_arr()
x,y = make_chart_arr()

min_price = min(y)
max_price = max(y)

date_time = datetime.datetime.fromtimestamp(x[0]/1000)
print("from=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

date_time = datetime.datetime.fromtimestamp(x[-1]/1000)
print("to=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

print("min_price=", min_price)
print("max_price=", max_price)


In [ ]:
day_x=np.arange(0,len(x),dtype=float)
IDX_IN_DAY = CHART_PER_DAY*16
day_x /= IDX_IN_DAY


In [ ]:
plt.plot(day_x,y)
#plt.xlim(20, 190)
#plt.ylim(0, 80000)
plt.show()


In [ ]:
byeMoneyAmmount = 100
sellRatio = 20050/20000
refinanceRatio = 1.5
flatRatio = 21000/20000

orders = []

last_time_bye = 0
last_sell_price = 0

def fewDaysFlat(idx,price):
    day_idx = int(idx/CHART_PER_DAY)
    
    if day_idx<6:
        return False
    
    max_price = price
    min_price = price
    
    for i in range(day_idx-7,day_idx+1):
        d = day_arr[i]
        max_price = max(max_price,float(d[2]))
        min_price = min(max_price,float(d[3]))
    
    return min_price*flatRatio>=max_price
    

def isTimeToBye(idx,price):
    global last_time_bye
    global last_sell_price
    
    search_price = False
    if len(orders) ==0:
        search_price = True
    elif len(orders[-1])>2:
        search_price = True

    if search_price:
        if last_sell_price !=0 and price*sellRatio < last_sell_price:
            return True;
        if last_time_bye == 0 or last_time_bye+CHART_PER_DAY <=idx:
            last_time_bye = idx
            return True
    else:
        prev_price = orders[-1][1]/sellRatio
        
        if price*refinanceRatio < prev_price and fewDaysFlat(idx,price):
            orders[-1].append(True)
            print("will refinance: price=",price,"for order_buy on price",prev_price,"len(orders)", len(orders))
            return True
    
    return False

In [ ]:
bye_points = []
sell_points = []
loss_points = []

order_points = []
order_money = []

SEARCH_BUY = 1
BUY = 2
    
money = 0

last_time_bye = 0
last_sell_price = 0

state = SEARCH_BUY
limitPrice = 0

bye_points = []
sell_points = []

order_points = []
order_money = []
orders = []

next_random = int(random.random()*16)

def push_amount(day,price):
    global money
    
    order_points.append(day);
    
    m = money;
    for o in orders:
        m += o[0]*price
    
    order_money.append(m);


for i in range(4,len(y)):
    price = y[i]
    
    sell_something = False
    
    for j in range(len(orders)-1,-1,-1) :
        o = orders[j]
        if price >= o[1]:
            money += o[0] * price
            sell_something = True
            orders.pop(j)

    if sell_something:
        push_amount(day_x[i],price)
        sell_points.append(day_x[i])
        last_sell_price = price
        #print(i,i/IDX_IN_DAY,"sell: price=",price,"money=",order_money[-1],"len(orders)",len(orders))
        

    if state == SEARCH_BUY:
        if (i%16)==next_random and isTimeToBye(int(i/16),price):
            next_random = int(random.random()*16)
            state = BUY
    elif state == BUY:
        if money > byeMoneyAmmount:
            btc = money/price
            money = 0
        else:
            m = byeMoneyAmmount*2^len(orders)
            btc = m/price
            money -= m
            
        order = [btc,price*sellRatio]
        orders.append(order)

        push_amount(day_x[i],price)
        bye_points.append(day_x[i])
        
        #print(i,i/IDX_IN_DAY,"buy: price=",price,"money=",order_money[-1],"will_sell_on=",order[1])
        state  = SEARCH_BUY



push_amount(day_x[i],price)

                
            
        
            
        




In [ ]:
print("money=",order_money[-1])
print("bye_points:",len(bye_points),"sell_points:",len(sell_points), "len(orders)", len(orders))

In [ ]:
plt.plot(day_x,y)

for p in bye_points:
    plt.vlines(p,min_price,max_price,color='green')

for p in sell_points:
    plt.vlines(p,min_price,max_price,color='red')

plt.show()

In [ ]:
fig, ax1 = plt.subplots()
color1 ='blue'
ax1.plot(day_x,y, color = color1)
ax1.set_ylabel('Price',color = color1);
ax1.tick_params(axis='y',color = color1);

color2 ='crimson'
ax2 = ax1.twinx();
ax2.plot(order_points,order_money, color = color2)
ax2.set_ylabel('Money', color = color2);
ax2.tick_params(axis='y',color = color2);


In [ ]:
print("min_money",min(order_money))
print("max_money",max(order_money))
print("money=",order_money[-1])